<a href="https://colab.research.google.com/github/xelothi/ML-for-kinase-inhibitors-development/blob/main/30_selecteed_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install padelpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 22.2 MB/s eta 0:00:00


In [2]:
! wget https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
! unzip fingerprints_xml.zip

--2024-03-30 10:47:05--  https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip [following]
--2024-03-30 10:47:05--  https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10871 (11K) [application/zip]
Saving to: ‘fingerprints_xml.zip’

fingerprints_xml.zi 100%[===================>]  10.62K  --.-KB/s    in 0s      

2024-03-30 10:47:05 (43.0 MB/s) - ‘fingerprints_xml.zip’ saved [10871/10871]

Archive:  fingerprints_xm

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
import glob
import os
from collections import defaultdict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from collections import defaultdict
from matplotlib.ticker import MultipleLocator
import statsmodels.api as sm
from copy import deepcopy
from statsmodels.stats.outliers_influence import summary_table
from tabulate import tabulate
import pickle

In [46]:
df = pd.read_csv("CRAF_train.csv", index_col=[0])
df = df.dropna()
df.rename(columns={"molecule_chembl_id": "Name"}, inplace=True)

In [47]:
atom_pairs_2D_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/AtomPairs2D_normalized.csv", sep=',')
atom_pairs_2D_count_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/AtomPairs2DCount_normalized.csv", sep=',')
CDK_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/CDK_normalized.csv", sep=",")
CDK_extended_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/CDKextened_normalized.csv", sep=",")
CDK_graphonly_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/CDK_graphonly_normalized.csv", sep=",")
EState_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/EState_normalized.csv", sep=",")
Klekota_roth_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/KlekotaRoth_nomralized.csv", sep=",")
Klekota_roth_count_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/KlekotaRothCount_normalized.csv", sep=",")
PubChem_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/PubChem_normalized.csv", sep=",")
substrucutre_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/Substructure_normalized.csv", sep=",")
substrucutre_count_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/SubstrucutreCount_normalized.csv", sep=",")
MACCS_norm = pd.read_csv("/content/gdrive/MyDrive/CRAF_hand_picked/MACCS_normalized.csv", sep=",")

In [48]:
cdk = CDK_norm[['FP408', 'FP389', 'FP293', 'FP382', 'FP637', 'FP359', 'FP1005', 'FP317', 'FP966', 'FP195','FP744', 'FP909', 'FP648', 'FP954', 'FP626', 'FP337', 'FP512', 'FP514', 'FP836', 'FP609', 'FP754', 'FP392', 'FP547', 'FP56', 'FP321', 'FP72', 'FP872', 'FP264', 'FP969', 'FP774']]
estate = EState_norm[['EStateFP30', 'EStateFP28', 'EStateFP9', 'EStateFP18', 'EStateFP38', 'EStateFP51', 'EStateFP36', 'EStateFP19', 'EStateFP11', 'EStateFP35', 'EStateFP25', 'EStateFP54', 'EStateFP13', 'EStateFP29', 'EStateFP7', 'EStateFP16', 'EStateFP53', 'EStateFP32', 'EStateFP24', 'EStateFP37', 'EStateFP21', 'EStateFP70', 'EStateFP15', 'EStateFP17', 'EStateFP8', 'EStateFP75']]
atom_pairs_2D = atom_pairs_2D_norm[['AD2D481', 'AD2D170', 'AD2D407', 'AD2D638', 'AD2D716', 'AD2D336', 'AD2D726', 'AD2D6', 'AD2D171', 'AD2D637', 'AD2D715', 'AD2D482', 'AD2D492', 'AD2D248', 'AD2D485', 'AD2D559', 'AD2D414', 'AD2D404', 'AD2D329', 'AD2D337', 'AD2D325', 'AD2D14', 'AD2D91', 'AD2D247', 'AD2D560', 'AD2D81', 'AD2D571', 'AD2D13', 'AD2D403', 'AD2D169']]
atom_pairs_2D_count = atom_pairs_2D_count_norm[['APC2D1_C_C', 'APC2D9_C_N', 'APC2D1_C_N', 'APC2D8_C_O', 'APC2D3_N_O', 'APC2D9_N_O', 'APC2D1_C_O', 'APC2D2_N_O', 'APC2D10_N_O', 'APC2D7_C_N', 'APC2D1_C_F', 'APC2D2_N_N', 'APC2D4_N_O', 'APC2D10_N_N', 'APC2D3_O_S', 'APC2D9_N_N', 'APC2D10_O_O', 'APC2D5_O_O', 'APC2D7_N_O', 'APC2D6_O_O', 'APC2D6_N_O', 'APC2D5_N_O', 'APC2D5_N_N', 'APC2D8_N_O', 'APC2D3_N_S', 'APC2D7_N_F', 'APC2D7_O_O', 'APC2D1_C_S', 'APC2D5_N_F', 'APC2D6_N_N']]
CDK_graphonly = CDK_graphonly_norm[['GraphFP837', 'GraphFP232', 'GraphFP200', 'GraphFP591', 'GraphFP251', 'GraphFP253', 'GraphFP144', 'GraphFP695', 'GraphFP21', 'GraphFP204', 'GraphFP481', 'GraphFP564', 'GraphFP575', 'GraphFP96', 'GraphFP910', 'GraphFP780', 'GraphFP277', 'GraphFP118', 'GraphFP523', 'GraphFP250', 'GraphFP551', 'GraphFP522', 'GraphFP135', 'GraphFP833', 'GraphFP905', 'GraphFP175', 'GraphFP268', 'GraphFP101', 'GraphFP178', 'GraphFP255']]
CDK_extended = CDK_extended_norm[['ExtFP612', 'ExtFP722', 'ExtFP301', 'ExtFP1013', 'ExtFP572', 'ExtFP279', 'ExtFP143', 'ExtFP716', 'ExtFP84', 'ExtFP217', 'ExtFP529', 'ExtFP851', 'ExtFP903', 'ExtFP397', 'ExtFP56', 'ExtFP866', 'ExtFP281', 'ExtFP406', 'ExtFP384', 'ExtFP370', 'ExtFP337', 'ExtFP99', 'ExtFP561', 'ExtFP434', 'ExtFP801', 'ExtFP964', 'ExtFP662', 'ExtFP220', 'ExtFP682', 'ExtFP422']]
Klekota_roth = Klekota_roth_norm[['KRFP444', 'KRFP839', 'KRFP1148', 'KRFP1438', 'KRFP297', 'KRFP1645', 'KRFP1642', 'KRFP3013', 'KRFP1524', 'KRFP1591', 'KRFP2667', 'KRFP1156', 'KRFP1947', 'KRFP1909', 'KRFP3943', 'KRFP2547', 'KRFP383', 'KRFP4269', 'KRFP1154', 'KRFP3369', 'KRFP3956', 'KRFP1750', 'KRFP2259', 'KRFP3224', 'KRFP2024', 'KRFP3640', 'KRFP810', 'KRFP1013', 'KRFP3008', 'KRFP4000']]
Klekota_roth_count_norm = Klekota_roth_count_norm[['KRFPC297', 'KRFPC839', 'KRFPC3574', 'KRFPC1148', 'KRFPC1438', 'KRFPC1645', 'KRFPC669', 'KRFPC3013', 'KRFPC1642', 'KRFPC677', 'KRFPC4331', 'KRFPC2547', 'KRFPC1524', 'KRFPC2667', 'KRFPC1591', 'KRFPC1909', 'KRFPC3943', 'KRFPC1947', 'KRFPC1154', 'KRFPC4269', 'KRFPC3751', 'KRFPC2259', 'KRFPC383', 'KRFPC3740', 'KRFPC2986', 'KRFPC1750', 'KRFPC2024', 'KRFPC1564', 'KRFPC1', 'KRFPC810']]
substrucutre = substrucutre_norm[['SubFP16', 'SubFP143', 'SubFP100', 'SubFP88', 'SubFP9', 'SubFP85', 'SubFP179', 'SubFP2', 'SubFP1', 'SubFP19', 'SubFP4', 'SubFP135', 'SubFP18', 'SubFP3', 'SubFP137', 'SubFP183', 'SubFP171', 'SubFP23', 'SubFP287', 'SubFP169', 'SubFP180', 'SubFP172', 'SubFP181', 'SubFP182', 'SubFP209', 'SubFP84', 'SubFP173', 'SubFP33', 'SubFP5', 'SubFP133']]
substrucutre_count = substrucutre_count_norm[['SubFPC16', 'SubFPC143', 'SubFPC300', 'SubFPC137', 'SubFPC275', 'SubFPC295', 'SubFPC302', 'SubFPC1', 'SubFPC88', 'SubFPC100', 'SubFPC2', 'SubFPC85', 'SubFPC9', 'SubFPC287', 'SubFPC181', 'SubFPC179', 'SubFPC183', 'SubFPC19', 'SubFPC3', 'SubFPC171', 'SubFPC23', 'SubFPC135', 'SubFPC18', 'SubFPC169', 'SubFPC172', 'SubFPC182', 'SubFPC180', 'SubFPC173', 'SubFPC209', 'SubFPC84']]
MACCS = MACCS_norm[['MACCSFP92', 'MACCSFP86', 'MACCSFP57', 'MACCSFP62', 'MACCSFP37', 'MACCSFP75', 'MACCSFP98', 'MACCSFP111', 'MACCSFP131', 'MACCSFP139', 'MACCSFP100', 'MACCSFP95', 'MACCSFP105', 'MACCSFP145', 'MACCSFP83', 'MACCSFP113', 'MACCSFP78', 'MACCSFP94', 'MACCSFP42', 'MACCSFP87', 'MACCSFP149', 'MACCSFP160', 'MACCSFP150', 'MACCSFP74', 'MACCSFP104', 'MACCSFP144', 'MACCSFP123', 'MACCSFP91', 'MACCSFP38', 'MACCSFP80']]
PubChem = PubChem_norm[['PubchemFP187', 'PubchemFP439', 'PubchemFP614', 'PubchemFP566', 'PubchemFP180', 'PubchemFP391', 'PubchemFP559', 'PubchemFP734', 'PubchemFP308', 'PubchemFP716', 'PubchemFP185', 'PubchemFP580', 'PubchemFP143', 'PubchemFP341', 'PubchemFP714', 'PubchemFP528', 'PubchemFP696', 'PubchemFP23', 'PubchemFP334', 'PubchemFP380', 'PubchemFP697', 'PubchemFP420', 'PubchemFP721', 'PubchemFP405', 'PubchemFP737', 'PubchemFP186', 'PubchemFP493', 'PubchemFP357', 'PubchemFP372', 'PubchemFP643']]

In [49]:
cdk.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/cdk.csv", index=False)
estate.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/estate.csv", index=False)
atom_pairs_2D.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/atom_pairs_2D.csv", index=False)
atom_pairs_2D_count.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/atom_pairs_2D_count.csv", index=False)
CDK_graphonly.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/CDK_graphonly.csv", index=False)
CDK_extended.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/CDK_extended.csv", index=False)
Klekota_roth.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/klekota_roth.csv", index=False)
Klekota_roth_count_norm.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/klekota_roth_count.csv", index=False)
substrucutre.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/substrucutre.csv", index=False)
substrucutre_count.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/substrucutre_count.csv", index=False)
MACCS.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/MACCS.csv", index=False)
PubChem.to_csv("/content/gdrive/MyDrive/CRAF_selected_features/PubChem.csv", index=False)

In [50]:
def extract_file_name(file_path):
    # Extract the file name without extension
    file_name = os.path.basename(file_path)
    file_name_without_extension = os.path.splitext(file_name)[0]
    return file_name_without_extension

In [51]:
def model_performance(y_train,y_test, prediction, cv, prediction_external):
    R2_train.append(r2_score(y_train, prediction))
    RMSE_train.append(np.sqrt(mean_absolute_error(y_train, prediction)))
    Q2_cross_val.append(r2_score(y_train, cv))
    RMSE_cross_val.append(np.sqrt(mean_absolute_error(y_train, cv)))
    Q2_ext.append(r2_score(y_test, prediction_external))
    RMSE_ext.append((mean_absolute_error(y_test, prediction_external)))


In [58]:
def perform_y_scrambling(X_train, X_test, y_train, y_test):

    for random_seed_counter in range(1, 101):
        # Create shuffled copies of the training labels and features
        y_train_shuffled = deepcopy(y_train)

        np.random.shuffle(y_train_shuffled)

        # Train a RandomForestRegressor on the shuffled data
        rf_scrambled = RandomForestRegressor()
        rf_scrambled = rf_scrambled.fit(X_train, y_train_shuffled)

        # Predictions on the test set
        y_predict_test = rf_scrambled.predict(X_test)
        r2_test_scramble = (rf_scrambled.score(X_test, y_test))**2
        r2_scores_test_scrambled.append(r2_test_scramble)

        # Predictions on the train set
        y_predict_train = rf_scrambled.predict(X_train)
        r2_train_scramble =  (rf_scrambled.score(X_train, y_train))**2
        r2_scores_train_scrambled.append(r2_train_scramble)

    return r2_scores_test_scrambled, r2_scores_train_scrambled

In [60]:
def metrics(R2_train, RMSE_train, Q2_CV, RMSE_CV, Q2_External, RMSE_External, importances_dict):
    R2_train_mean = np.mean(R2_train)
    RMSE_train_mean = np.mean(RMSE_train)
    Q2_CV_mean = np.mean(Q2_CV)




    RMSE_CV_mean = np.mean(RMSE_CV)
    Q2_External_mean = np.mean(Q2_External)
    RMSE_External_mean = np.mean(RMSE_External)

    R2_train_std = np.std(R2_train)
    RMSE_train_std = np.std(RMSE_train)
    Q2_CV_std = np.std(Q2_CV)
    RMSE_CV_std = np.std(RMSE_CV)
    Q2_External_std = np.std(Q2_External)
    RMSE_External_std = np.std(RMSE_External)


    importances_mean0 = {}
    for fx in importances_dict:
        importances_mean0[fx] = np.mean(importances_dict[fx])
    importances_mean = sorted([(k,v) for k,v in importances_mean0.items()],
                                     key=lambda x: x[1], reverse=True)

    importances_std0 = {}
    for fx in importances_dict:
        importances_std0[fx] = np.std(importances_dict[fx])
    importances_std = sorted([(k,v) for k,v in importances_std0.items()],
                                     key=lambda x: x[1], reverse=True)


    return R2_train_mean, RMSE_train_mean, Q2_CV_mean, RMSE_CV_mean, Q2_External_mean, RMSE_External_mean, R2_train_std, RMSE_train_std, Q2_CV_std, RMSE_CV_std, Q2_External_std, RMSE_External_std, importances_mean, importances_std

In [52]:
def model_training_and_validation(X,Y,seed, hx, verbose=True):


    #Data split using 70/30 ratio
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        Y, test_size=0.2,
                                                        random_state=seed)
    param = {
        'max_depth': 20,
        'max_features': 'sqrt',
        'min_samples_leaf': 1,
        'min_samples_split': 2,
        'n_estimators': 200,
        "n_jobs": -1
    }
    model_RF = RandomForestRegressor(random_state=SEED, **param)

    model_RF.fit(X_train,y_train)
    prediction = model_RF.predict(X_train)

    # Cross-validation
    cv = cross_val_predict(model_RF, X_train, y_train, cv=10, n_jobs=-1)

    # External set
    prediction_external = model_RF.predict(X_test)

    #print result from each seed
    model_performance(y_train,y_test, prediction, cv, prediction_external)

    #Feature Importance
    Feature = hx[:]
    feature_importance = model_RF.feature_importances_
    importances = 100.0 * (feature_importance / feature_importance.max()) #index

    for i, fx in enumerate(Feature):
        importances_dict[str(fx)].append(importances[i])

    pickle.dump(model_RF, open(f'{model_name}.pkl_', 'wb'))
    return Feature, X_train, X_test, y_train, y_test, importances_dict

In [53]:
path = "/content/gdrive/MyDrive/CRAF_selected_features/"

In [63]:
from tabulate import tabulate

SEED = 13

for fp_file in glob.glob(f'{path}*.csv'):
    # Extract the file name without extension
    model_name = extract_file_name(fp_file)
    print("----------------------------------------------------------------------------------------------")
    print(model_name)
    print("---------------------------------------------------")


    df_fp = pd.read_csv(fp_file)
    data = df_fp.iloc[:, :]
    Y = df["pIC50"].values.astype(float)


    hx = np.array(h)

    data = data.values.astype(float)
    X = np.array(data)

    # Prepare empty lists to plot QSAR model
    R2_train = []
    RMSE_train = []
    Q2_cross_val = []
    RMSE_cross_val = []
    Q2_ext = []
    RMSE_ext = []
    prediction_train = []
    importances_dict = defaultdict(list)

    # Prepare empty lists to plot the performance of accuracy.
    r2_scores_train_scrambled= []
    r2_scores_test_scrambled = []



    # Perform model training and validation using the model_name
    for i in range(10):
         Feature, X_train, X_test, y_train, y_test,importances_dict = model_training_and_validation(X, Y, i, hx)

    r2_scores_train_scrambled, r2_scores_test_scrambled =  perform_y_scrambling(X_train, X_test, y_train, y_test)

    R2_train_mean, RMSE_train_mean, Q2_CV_mean, RMSE_CV_mean, Q2_External_mean, RMSE_External_mean, R2_train_std, RMSE_train_std, Q2_CV_std, RMSE_CV_std, Q2_External_std, RMSE_External_std, importances_mean, importances_std = metrics(R2_train, RMSE_train,Q2_cross_val,RMSE_cross_val, Q2_ext,RMSE_ext, importances_dict)

    metrics_table = [
    ['Metric', 'Value'],
    ['Training set', ''],
    ['N', f'{len(X_train)}'],
    ['R2', f'{R2_train_mean:.4f}'],
    ['std_R2', f'{R2_train_std:.6f}'],
    ['RMSE', f'{RMSE_train_mean:.4f}'],
    ['std_RMSE', f'{RMSE_train_std:.6f}'],
    ['', ''],
    ['Cross-validation set', ''],
    ['N', f'{len(X_train)}'],
    ['Q2', f'{Q2_CV_mean:.4f}'],
    ['std_Q2', f'{Q2_CV_std:.6f}'],
    ['RMSE', f'{RMSE_CV_mean:.4f}'],
    ['std_RMSE', f'{RMSE_CV_std:.6f}'],
    ['', ''],
    ['External set', ''],
    ['N', f'{len(X_test)}'],
    ['Q2_EXt', f'{Q2_External_mean:.4f}'],
    ['std_Q2_EXt', f'{Q2_External_std:.6f}'],
    ['RMSE', f'{RMSE_External_mean:.4f}'],
    ['std_RMSE', f'{RMSE_External_std:.6f}']]
    print(tabulate(metrics_table, headers='firstrow', tablefmt='grid'))




----------------------------------------------------------------------------------------------
substrucutre_count
---------------------------------------------------
+----------------------+----------+
| Metric               | Value    |
+======================+==========+
| Training set         |          |
+----------------------+----------+
| N                    | 472      |
+----------------------+----------+
| R2                   | 0.9492   |
+----------------------+----------+
| std_R2               | 0.004373 |
+----------------------+----------+
| RMSE                 | 0.4754   |
+----------------------+----------+
| std_RMSE             | 0.007720 |
+----------------------+----------+
|                      |          |
+----------------------+----------+
| Cross-validation set |          |
+----------------------+----------+
| N                    | 472      |
+----------------------+----------+
| Q2                   | 0.7497   |
+----------------------+----------+
| std_